In [2]:
import pandas as pd

In [ ]:
#create the sec codes dataset
sic_codes = pd.read_csv('corpwatch_api_tables_csv/sic_codes.csv', sep = "\t", escapechar='\\')
sic_sectors = pd.read_csv('corpwatch_api_tables_csv/sic_sectors.csv', sep = "\t", escapechar='\\')

In [ ]:
sic_sectors_codes = pd.merge(sic_codes,
                     sic_sectors,
                     on='sic_sector', 
                     how='left')

In [ ]:
sic_sectors_codes.to_csv("sector_output.csv")

## Read in the companies and merge with the codes

In [3]:
company_info = pd.read_csv('corpwatch_api_tables_csv/company_info.csv', sep = "\t", escapechar='\\',  low_memory=False)

In [ ]:
company_info.columns

In [ ]:
company_info.head()

In [4]:
recent_companies = company_info[company_info['most_recent'] == 1]

In [ ]:

# custom_aggregations = {'cw_id' : ['first'],
#                        'company_name': ['first', 'last'],
#                        'cik' : ['first'],
#                        'sic_code' : ['first'],
#                        'industry_name' : ['first'],
#                        'sic_sector' : ['first'],
#                        'sector_name' : ['first'],
#                        'num_parents' : ['max'],
#                        'num_children' : ['max'],
#                        'top_parent_id' : ['max'],
#                         'year' : ['min','count'],
#                        'max_year' : ['max'],
#                        'min_year' : ['min']
#                       }
# #this aggregation is not needed, cw_id is unique
# company_list = recent_companies.groupby('cw_id').agg(custom_aggregations)

In [7]:
recent_companies.head()

,row_id,cw_id,most_recent,year,cik,irs_number,best_location_id,sic_code,industry_name,sic_sector,sector_name,source_type,source_id,num_parents,num_children,top_parent_id,company_name,max_year,min_year,no_sic
17,18,37,1,2019,3153,630004250.0,74,4911.0,Electric services,4900.0,"Electric, gas, and sanitary services",filers,1468956,5,101,37,ALABAMA POWER CO,2019,2003,0
23,24,45,1,2019,3499,510100517.0,88,6798.0,Real estate investment trusts,6700.0,Holding and other investment offices,filers,1446212,2,23,9019,ALEXANDERS INC,2019,2003,0
38,39,69,1,2019,4515,131502798.0,137,4512.0,"Air transportation, scheduled",4500.0,Transportation by air,filers,1436980,1,25,105,AMERICAN AIRLINES INC,2019,2003,0
64,65,120,1,2019,6879,540124790.0,246,4911.0,Electric services,4900.0,"Electric, gas, and sanitary services",filers,1469963,7,21,75,APPALACHIAN POWER CO,2019,2003,0
78,79,130,1,2019,7323,831918668.0,273,4911.0,Electric services,4900.0,"Electric, gas, and sanitary services",filers,1454053,1,3,1180,ENTERGY ARKANSAS LLC,2019,2003,0


## Read in relationships, create roll up of parent-child and child-parent dataset

In [8]:
relationships = pd.read_csv('corpwatch_api_tables_csv/relationships.csv', sep = "\t", escapechar='\\')

In [ ]:
#company_relationships = pd.read_csv('corpwatch_api_tables_csv/company_relations.csv', sep = "\t", escapechar='\\')

In [11]:
relationships_recent = relationships[relationships['year'] > 2018]

In [12]:
relationships_recent.head()

,relationship_id,company_name,location,filing_id,country_code,subdiv_code,clean_company,cik,ignore_record,parse_method,hierarchy,percent,parent_cw_id,cw_id,filer_cik,year,quarter
4094806,4094991,"PolarityTE, Inc",Nevada,16555966,US,NV,PolarityTE Inc,0.0,0,"simple table, company|location",0,NaN,16876.0,1091408.0,1076682,2019,1
4094807,4094992,"PolarityTE MD, Inc",Nevada,16555966,US,NV,PolarityTE MD Inc,0.0,0,"simple table, company|location",0,NaN,16876.0,1091409.0,1076682,2019,1
4094808,4094993,"PolarityTE RD, Inc",Nevada,16555966,US,NV,PolarityTE RD Inc,0.0,0,"simple table, company|location",0,NaN,16876.0,1091410.0,1076682,2019,1
4094809,4094994,"Utah CRO Services, Inc",Nevada,16555966,US,NV,Utah CRO Services Inc,0.0,0,"simple table, company|location",0,NaN,16876.0,1091468.0,1076682,2019,1
4094810,4094995,"IBEX Preclinical Research, Inc",Nevada,16555966,US,NV,IBEX Preclinical Research Inc,0.0,0,"simple table, company|location",0,NaN,16876.0,1091329.0,1076682,2019,1


In [ ]:
# import matplotlib.pyplot as plt
# num= relationships_recent[relationships_recent['hierarchy'] > 0]
# plt.hist(num['hierarchy'], range = (0,50))

In [ ]:
# num_high = relationships_recent[relationships_recent['hierarchy'] > 20]
# semi = relationships_recent[relationships_recent['filer_cik'] == 103730]
# semi[1:10]

In [13]:
def unique_non_nan(elem):
    return [a for a in list(pd.unique(elem)) if pd.notna(a)]
unique_non_nan([5.0, float('nan')]) 
   

[5.0]

In [150]:
PARENT_CW_ID = "parent_cw_id"

parent_child_groups = relationships_recent.groupby(['parent_cw_id']).agg({
     'parent_cw_id': "first",
     'cw_id': unique_non_nan,
     'clean_company': unique_non_nan})

In [151]:
parent_child_groups.head()

,parent_cw_id,cw_id,clean_company
parent_cw_id,,,
2.0,2.0,"[2.0, 724193.0, 724210.0, 198289.0, 198295.0, ...","[AAR CORP, AAR do Brasil Ltda, Rags of Honor L..."
3.0,3.0,"[638746.0, 122844.0, 122845.0, 122846.0, 12284...","[Abbott Biologicals LLC, Abbott Cardiovascular..."
13.0,13.0,"[124888.0, 124889.0, 124890.0, 737227.0, 83464...","[Acme United Ltd, Acme United Europe GmbH, Acm..."
16.0,16.0,"[125412.0, 125414.0, 125415.0, 125416.0, 12542...","[Service Transport Co, Ada Crude Oil Co, Ada M..."
17.0,17.0,"[752074.0, 984690.0]","[RELM Communications Inc, Tactical Capital Inv..."


In [152]:
def deduplicate(col, ls):
    if col in ls: ls.remove(col)
    return ls


In [153]:
parent_child_groups.columns
#parent_child_groups['clean_child_list'] = parent_child_groups.apply(lambda row: row['cw_id'], axis = 1)
parent_child_groups['child_cw_id_list'] = parent_child_groups.apply(lambda row: deduplicate(row['parent_cw_id'], row.cw_id), axis = 1)

In [158]:
CHILD_COMPANY_NAMES = "child_company_names"
CHILD_CW_IDS = "child_cw_ids"
PARENT_CW_IDS = "parent_cw_ids"
PARENT_COMPANY_NAMES = "parent_company_names"

parent_child_groups = parent_child_groups.rename(
    columns= {
    "cw_id": CHILD_CW_IDS,
    'clean_company': CHILD_COMPANY_NAMES})
x = parent_child_groups.rename(index = {'parent_cw_id': "cw_id"}, inplace = False)
parent_child_groups = parent_child_groups[['child_cw_id_list']]

parent_child_groups.head()

,child_cw_id_list
parent_cw_id,
2.0,"[724193.0, 724210.0, 198289.0, 198295.0, 69637..."
3.0,"[638746.0, 122844.0, 122845.0, 122846.0, 12284..."
13.0,"[124888.0, 124889.0, 124890.0, 737227.0, 83464..."
16.0,"[125412.0, 125414.0, 125415.0, 125416.0, 12542..."
17.0,"[752074.0, 984690.0]"


In [131]:
parent_child_groups.head()

,cw_id,child_cw_id_list
parent_cw_id,,
2.0,2.0,"[724193.0, 724210.0, 198289.0, 198295.0, 69637..."
3.0,3.0,"[638746.0, 122844.0, 122845.0, 122846.0, 12284..."
13.0,13.0,"[124888.0, 124889.0, 124890.0, 737227.0, 83464..."
16.0,16.0,"[125412.0, 125414.0, 125415.0, 125416.0, 12542..."
17.0,17.0,"[752074.0, 984690.0]"


In [101]:
child_parent_groups = relationships_recent.groupby(['cw_id']).agg({
     'cw_id': "first",
     'parent_cw_id' : lambda x:  unique_non_nan(x),
     'clean_company': lambda x: unique_non_nan(x)})

In [102]:
child_parent_groups['parent_cw_id_list'] = child_parent_groups.apply(lambda row: deduplicate(row['cw_id'], row['parent_cw_id']), axis =1)

In [103]:
child_parent_groups.head()

,cw_id,parent_cw_id,clean_company,parent_cw_id_list
cw_id,,,,
2.0,2.0,[],[AAR CORP],[]
34.0,34.0,[],[Akorn Inc],[]
37.0,37.0,"[12009.0, 22569.0, 781.0, 1197.0, 1665.0]",[Alabama Power Co],"[12009.0, 22569.0, 781.0, 1197.0, 1665.0]"
45.0,45.0,"[14480.0, 9019.0]",[Alexanders Inc],"[14480.0, 9019.0]"
68.0,68.0,[],[AMERCO],[]


In [134]:
child_parent_groups = child_parent_groups[['parent_cw_id_list']]
child_parent_groups.head()                         

,parent_cw_id_list
cw_id,
2.0,[]
34.0,[]
37.0,"[12009.0, 22569.0, 781.0, 1197.0, 1665.0]"
45.0,"[14480.0, 9019.0]"
68.0,[]


In [165]:


# join the relationships with the originally company dataset
company_joined1 = pd.merge(recent_companies,
                     parent_child_groups, 
                     left_on= 'cw_id',
                     right_on='parent_cw_id',          
                     how='left')

In [166]:
company_joined1.head()

,row_id,cw_id,most_recent,year,cik,irs_number,best_location_id,sic_code,industry_name,sic_sector,...,source_type,source_id,num_parents,num_children,top_parent_id,company_name,max_year,min_year,no_sic,child_cw_id_list
0,18,37,1,2019,3153,630004250.0,74,4911.0,Electric services,4900.0,...,filers,1468956,5,101,37,ALABAMA POWER CO,2019,2003,0,"[1665.0, 993179.0, 101861.0, 1018259.0, 781.0,..."
1,24,45,1,2019,3499,510100517.0,88,6798.0,Real estate investment trusts,6700.0,...,filers,1446212,2,23,9019,ALEXANDERS INC,2019,2003,0,"[1038175.0, 1038176.0, 1038177.0, 1038178.0, 1..."
2,39,69,1,2019,4515,131502798.0,137,4512.0,"Air transportation, scheduled",4500.0,...,filers,1436980,1,25,105,AMERICAN AIRLINES INC,2019,2003,0,"[964843.0, 964842.0, 411832.0, 411833.0, 41612..."
3,65,120,1,2019,6879,540124790.0,246,4911.0,Electric services,4900.0,...,filers,1469963,7,21,75,APPALACHIAN POWER CO,2019,2003,0,"[993634.0, 834826.0, 696650.0, 992937.0, 13682..."
4,79,130,1,2019,7323,831918668.0,273,4911.0,Electric services,4900.0,...,filers,1454053,1,3,1180,ENTERGY ARKANSAS LLC,2019,2003,0,"[899487.0, 1001712.0, 1180.0]"


In [167]:
company_joined2 =  pd.merge(company_joined1,
                     child_parent_groups, 
                     left_on= 'cw_id',
                     right_on='cw_id',
                     how='left')

In [168]:
company_joined2.head()

,row_id,cw_id,most_recent,year,cik,irs_number,best_location_id,sic_code,industry_name,sic_sector,...,source_id,num_parents,num_children,top_parent_id,company_name,max_year,min_year,no_sic,child_cw_id_list,parent_cw_id_list
0,18,37,1,2019,3153,630004250.0,74,4911.0,Electric services,4900.0,...,1468956,5,101,37,ALABAMA POWER CO,2019,2003,0,"[1665.0, 993179.0, 101861.0, 1018259.0, 781.0,...","[12009.0, 22569.0, 781.0, 1197.0, 1665.0]"
1,24,45,1,2019,3499,510100517.0,88,6798.0,Real estate investment trusts,6700.0,...,1446212,2,23,9019,ALEXANDERS INC,2019,2003,0,"[1038175.0, 1038176.0, 1038177.0, 1038178.0, 1...","[14480.0, 9019.0]"
2,39,69,1,2019,4515,131502798.0,137,4512.0,"Air transportation, scheduled",4500.0,...,1436980,1,25,105,AMERICAN AIRLINES INC,2019,2003,0,"[964843.0, 964842.0, 411832.0, 411833.0, 41612...",[105.0]
3,65,120,1,2019,6879,540124790.0,246,4911.0,Electric services,4900.0,...,1469963,7,21,75,APPALACHIAN POWER CO,2019,2003,0,"[993634.0, 834826.0, 696650.0, 992937.0, 13682...","[956587.0, 1029138.0, 75.0, 927.0, 1345.0, 149..."
4,79,130,1,2019,7323,831918668.0,273,4911.0,Electric services,4900.0,...,1454053,1,3,1180,ENTERGY ARKANSAS LLC,2019,2003,0,"[899487.0, 1001712.0, 1180.0]",[590084.0]


In [ ]:
# import matplotlib.pyplot as plt
# num_children = company_joined[company_joined['num_children_max'] > -1]['num_children_max']
# plt.hist(num_children, bins=100, range=[0, 500])

In [ ]:
# num_children = company_joined[(company_joined['num_children_max'] < 10) & (company_joined['num_children_max'] > 0)]['num_children_max']
# plt.hist(num_children)